In [49]:
import pandas as pd
import os
import json
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [50]:
nyc_csv = os.path.join("..", "data", "rawdata", "final_coll_data_df.csv")

csv = pd.read_csv(nyc_csv, low_memory=False)

In [71]:
nyc_df = pd.DataFrame(csv)
nyc_df.head(2)

,UNIX DATE,YEAR,MONTH,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Stan_Date,Week,Year / Week,Year / Month
0,1546171200,2018,12,0.0,0.0,2018-12-30,52,2018/52,2018/12
1,1546171200,2018,12,0.0,0.0,2018-12-30,52,2018/52,2018/12


In [72]:
nyc_df['Stan_Date'] = pd.to_datetime(nyc_df['UNIX DATE'],unit='s')
nyc_df.head(2)

,UNIX DATE,YEAR,MONTH,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Stan_Date,Week,Year / Week,Year / Month
0,1546171200,2018,12,0.0,0.0,2018-12-30 12:00:00,52,2018/52,2018/12
1,1546171200,2018,12,0.0,0.0,2018-12-30 12:00:00,52,2018/52,2018/12


In [73]:
nyc_df['Week'] = nyc_df['Stan_Date'].dt.week
nyc_df['Year / Week'] = nyc_df['YEAR'].map(str) + "/" + nyc_df['Week'].map(str)
nyc_df['Year / Month'] = nyc_df['YEAR'].map(str) + "/" + nyc_df['MONTH'].map(str)
nyc_df['Stan_Date'] = nyc_df['Stan_Date'].dt.date
nyc_df = nyc_df.rename(columns={'Stan_Date': 'DATE'})
nyc_df = nyc_df.rename(columns={"NUMBER OF PERSONS INJURED":"INJURIES", "NUMBER OF PERSONS KILLED":"DEATHS"})
nyc_df.head(2)

,UNIX DATE,YEAR,MONTH,INJURIES,DEATHS,DATE,Week,Year / Week,Year / Month
0,1546171200,2018,12,0.0,0.0,2018-12-30,52,2018/52,2018/12
1,1546171200,2018,12,0.0,0.0,2018-12-30,52,2018/52,2018/12


In [64]:
nyc_df.columns

Index(['UNIX DATE', 'YEAR', 'MONTH', 'INJURIES', 'DEATHS', 'DATE', 'Week',
       'Year / Week', 'Year / Month'],
      dtype='object')

In [74]:
nyc = ['DATE', 'Year / Month', 'Year / Week', 'UNIX DATE', 'Week', 'MONTH', 'YEAR',
       'DEATHS', 'INJURIES']

In [75]:
nyc_df = nyc_df[nyc]
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES
0,2018-12-30,2018/12,2018/52,1546171200,52,12,2018,0.0,0.0
1,2018-12-30,2018/12,2018/52,1546171200,52,12,2018,0.0,0.0


In [76]:
nyc_df=nyc_df.sort_values('DATE', ascending=True)
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES
1116398,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0
1116081,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0


In [77]:
def f(row):
    if row['MONTH'] == 12 and row['Week'] == 1:
        val = row['YEAR']+1
    elif row['MONTH'] == 12 and row['Week'] == 53:
        val = row['YEAR']+1
    else:
        val = row['YEAR']
    return val

In [78]:
nyc_df['Corrected Year for Year / Week'] = nyc_df.apply(f, axis=1)

In [79]:
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week
1116398,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014
1116081,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0,2014


In [80]:
def f(row):
    if row['MONTH'] == 12 and row['Week'] == 53:
        val = row['Week']-52
    elif row['MONTH'] == 1 and row['Week'] == 53:
        val = row['Week']-52
    elif row['MONTH'] == 1 and row['Week'] == 52:
        val = row['Week']-51
    else:
        val = row['Week']
    return val

In [81]:
nyc_df['Corrected Week for Year / Week'] = nyc_df.apply(f, axis=1)

In [82]:
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week,Corrected Week for Year / Week
1116398,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014,1
1116081,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0,2014,1


In [83]:
nyc_df['Corrected Year / Week'] = nyc_df['Corrected Year for Year / Week'].map(str) + "/" + nyc_df['Corrected Week for Year / Week'].map(str)

In [84]:
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week,Corrected Week for Year / Week,Corrected Year / Week
1116398,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014,1,2014/1
1116081,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0,2014,1,2014/1


In [85]:
nyc_df.columns

Index(['DATE', 'Year / Month', 'Year / Week', 'UNIX DATE', 'Week', 'MONTH',
       'YEAR', 'DEATHS', 'INJURIES', 'Corrected Year for Year / Week',
       'Corrected Week for Year / Week', 'Corrected Year / Week'],
      dtype='object')

In [88]:
nyc2 = ['DATE', 'Year / Month', 'Year / Week', 'UNIX DATE', 'Week', 'MONTH',
       'YEAR', 'DEATHS', 'INJURIES', 'Corrected Year for Year / Week',
       'Corrected Week for Year / Week', 'Corrected Year / Week']

In [89]:
nyc_df = nyc_df[nyc2]
nyc_df.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week,Corrected Week for Year / Week,Corrected Year / Week
1116398,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014,1,2014/1
1116081,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0,2014,1,2014/1


In [90]:
dark_sky_csv1 = "../data/rawdata/dark_sky_1.csv"
dark_sky_csv2 = "../data/rawdata/dark_sky_2.csv"
dark_sky_csv3 = "../data/rawdata/dark_sky_3.csv"

In [91]:
dark_sky_1 = pd.read_csv(dark_sky_csv1)
dark_sky_2 = pd.read_csv(dark_sky_csv2)
dark_sky_3 = pd.read_csv(dark_sky_csv3)

In [92]:
frames = [dark_sky_1, dark_sky_2, dark_sky_3]
dark_sky_df = pd.concat(frames, ignore_index=True)
dark_sky_df.reset_index
dark_sky_df.head(2)

,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,precipIntensity,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,0.0,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
1,1388599200,partly-cloudy-day,0.02,Mostly cloudy throughout the day.,1388578876,1388612417,0.44,14.62,0.56,0.0,...,0.0,1027.54,32.61,1388606400,23.02,1388577600,10.00,10.02,1388552400,1.43


In [93]:
len(dark_sky_df)

1912

In [94]:
dark_sky_df.drop_duplicates(subset=['Day'], keep= 'first', inplace = True)
len(dark_sky_df)

1910

In [95]:
dark_sky_df['Stan_Date'] = pd.to_datetime(dark_sky_df['Day'],unit='s')
dark_sky_df.head(1)

,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,precipIntensity,...,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed,Stan_Date
0,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,0.0,...,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76,2013-12-31 18:00:00


In [96]:
dark_sky_df['Stan_Date'] = dark_sky_df['Stan_Date'].dt.date
dark_sky_df.head(1)

,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,precipIntensity,...,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed,Stan_Date
0,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,0.0,...,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76,2013-12-31


In [97]:
dark_sky_df.columns

Index(['Day', 'Detail', 'Moonphase', 'Summary', 'Sunrise', 'Sunset',
       'cloudCover', 'dewPoint', 'humidity', 'precipIntensity',
       'precipIntensityMax', 'precipProbability', 'pressure', 'temperatureMax',
       'temperatureMaxTime', 'temperatureMin', 'temperatureMinTime',
       'visibility', 'windGust', 'windGustTime', 'windSpeed', 'Stan_Date'],
      dtype='object')

In [98]:
dark_sky = ['Stan_Date' , 'Day', 'Detail', 'Moonphase', 'Summary', 'Sunrise', 'Sunset',
       'cloudCover', 'dewPoint', 'humidity', 'precipIntensity',
       'precipIntensityMax', 'precipProbability', 'pressure', 'temperatureMax',
       'temperatureMaxTime', 'temperatureMin', 'temperatureMinTime',
       'visibility', 'windGust', 'windGustTime', 'windSpeed']

In [99]:
dark_sky_df = dark_sky_df[dark_sky]
dark_sky_df.head(1)

,Stan_Date,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,2013-12-31,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76


In [100]:
dark_sky_df = dark_sky_df.rename(columns={'Stan_Date': 'DATE'})
dark_sky_df.head(1)

,DATE,Day,Detail,Moonphase,Summary,Sunrise,Sunset,cloudCover,dewPoint,humidity,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,2013-12-31,1388512800,partly-cloudy-day,0.97,Mostly cloudy until evening.,1388492470,1388525969,0.47,12.98,0.57,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76


In [101]:
coll_n_weather_chris_stan_date = pd.merge(nyc_df, dark_sky_df, on="DATE", how='left')
coll_n_weather_chris_stan_date.head(2)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
1,2013-12-31,2013/12,2013/1,1388499660,1,12,2013,0.0,1.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76


In [102]:
len(coll_n_weather_chris_stan_date)

1116399

In [103]:
coll_n_weather_chris_stan_date=coll_n_weather_chris_stan_date.sort_values('DATE', ascending=True)
coll_n_weather_chris_stan_date.head(50)

,DATE,Year / Month,Year / Week,UNIX DATE,Week,MONTH,YEAR,DEATHS,INJURIES,Corrected Year for Year / Week,...,precipProbability,pressure,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,visibility,windGust,windGustTime,windSpeed
0,2013-12-31,2013/12,2013/1,1388483880,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
316,2013-12-31,2013/12,2013/1,1388483640,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
315,2013-12-31,2013/12,2013/1,1388483100,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
314,2013-12-31,2013/12,2013/1,1388482800,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
313,2013-12-31,2013/12,2013/1,1388482500,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
312,2013-12-31,2013/12,2013/1,1388482200,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
311,2013-12-31,2013/12,2013/1,1388482200,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
310,2013-12-31,2013/12,2013/1,1388482200,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
309,2013-12-31,2013/12,2013/1,1388482200,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76
308,2013-12-31,2013/12,2013/1,1388481900,1,12,2013,0.0,0.0,2014,...,0.0,1021.22,31.36,1388520000,20.95,1388491200,9.97,15.22,1388534400,2.76


In [104]:
coll_n_weather_chris_stan_date.columns

Index(['DATE', 'Year / Month', 'Year / Week', 'UNIX DATE', 'Week', 'MONTH',
       'YEAR', 'DEATHS', 'INJURIES', 'Corrected Year for Year / Week',
       'Corrected Week for Year / Week', 'Corrected Year / Week', 'Day',
       'Detail', 'Moonphase', 'Summary', 'Sunrise', 'Sunset', 'cloudCover',
       'dewPoint', 'humidity', 'precipIntensity', 'precipIntensityMax',
       'precipProbability', 'pressure', 'temperatureMax', 'temperatureMaxTime',
       'temperatureMin', 'temperatureMinTime', 'visibility', 'windGust',
       'windGustTime', 'windSpeed'],
      dtype='object')

In [105]:
coll_n_weather_chris_stan_date.to_csv("../data/cleandata/coll_n_weather_chris_stan_date_left_merge_period_intervals.csv", index = False, header = True)